In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import sqlite3
import mysql.connector as cnt
import plotly.express as px 
import requests
# import selenium
import os

# from bs4 import BeautifulSoup

In [3]:
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium import webdriver
from bs4 import BeautifulSoup
import time

In [4]:
opcijos = Options()
opcijos.add_argument('--incognito')
driver = webdriver.Chrome(options=opcijos)
url = 'https://www.varle.lt/dronai/'
driver.get(url)
time.sleep(5)
source = driver.page_source
driver.close()

In [5]:
bs = BeautifulSoup(source, 'html.parser')

In [6]:
sk = bs.find('div', {'class':'GRID_ITEM'}).find('a')
print(sk)
print(':::NEXT:::')
print(sk['href'])
img = sk.find('img')
print(img['height'])

<a href="/dronai/dronas-dji-mini-2-se-fly-more-combo--23881156.html">
<img alt="Nuotrauka Nr.1" class="product-img primary-img" height="195" loading="lazy" src="https://www.varle.lt/static/uploads/products/235x195/404/bun/bundle-fly-more-dji-mini-2-se-mini-drone.jpg" title="Dronas DJI Mini 2 SE Fly More Combo" width="235"/>
<img alt="Nuotrauka Nr.2" class="product-img secondary-img" height="195" loading="lazy" src="https://www.varle.lt/static/uploads/products/235x195/404/bun/bundle-fly-more-dji-mini-2-se-mini-drone_Wo609zb.jpg" title="Dronas DJI Mini 2 SE Fly More Combo" width="235"/>
</a>
:::NEXT:::
/dronai/dronas-dji-mini-2-se-fly-more-combo--23881156.html
195


In [7]:
# Surinkti iš kauno.diena.lt puslapio straipsnių pavadiniumų ilgius,
# tam panaudoti selenium biblioteką. Surinktus duomenis surašyti į SQLite duomenų bazę.
# Surašoma infromacija - pavadinimas, nuoroda į straipsnį, žodžių kiekis pavadinime. 
# SQLite lentelėje trys stulpeliai.

opcijos = Options()
opcijos.add_argument('--incognito')
driver = webdriver.Chrome(options=opcijos)
url = 'https://kauno.diena.lt/'
driver.get(url)
time.sleep(5)
source = driver.page_source
driver.close()

In [8]:



bs_kauno = BeautifulSoup(source, 'html.parser')
# print(bs)
nuorodos_kauno = []
pavadinimai_kauno = []
straipsniai_kauno = bs_kauno.find_all('div', {'category-block-row grid-12 alpha omega unscore'})
for s in straipsniai_kauno:
    i = s.find('div', {'class':'articles-list-title-wrapper'})
    if i is not None:
        t = i.find('a', {'class':'articles-list-title'})
        pavadinimai_kauno.append(t.text)
        nuorodos_kauno.append(t['href'])
        # print(':::REM:::')
        # print(t)
        # print('HREF?')
        # print(t['href'])
        # print(t.text)

print(len(pavadinimai_kauno))
print(pavadinimai_kauno)
print(len(nuorodos_kauno))
print(nuorodos_kauno)

29
['2024-ųjų Seimo rinkimai: laimėjo socdemai, antri – TS-LKD, treti – „Nemuno Aušra“ ', 'Į antrąjį Seimo rinkimų turą patekę kandidatai (apžvalga)', 'Judrioje Aleksoto gatvėje – laikini eismo ribojimai', 'Septyniose apygardose bus perskaičiuoti balsai, Panemunėje gali būti trys kandidatai ', 'Kauniečiai noriai balsavo Seimo rinkimuose: kai kur – stebinantis aktyvumas', 'Vytėnų gyventojas: šviesoforas neveikia, neįmanoma išvažiuoti', '„Lidl Lietuva“: ir toliau didinsime parduotuvių bei logistikos centro darbuotojų atlyginimus', 'Akibrokštas privačiai greitajai pagalbai: 50 įdarbintų žmonių valstybei neįdomu?', 'Vyriausybė nepritaria, kad baigę gyvulininkystę teiktų sveikatos paslaugas su gyvūnais', 'Šeima – tarsi universitetas, kuriame mokomės iš artimųjų', 'Ką pasirinks L. Mažylis – Seimą ar EP?', 'L. Kasčiūno rinkimuose aplenktas G. Landsbergis: tai – nueinančio pirmininko efektas ', 'Iš išaugtų vaikiškų drabužių – prisiminimų šiltukai', 'Mainų platformos įkūrėja: gyventi tvariau – 

In [9]:
zodziu_kiekiai_kauno = []
for pav in pavadinimai_kauno:
    kiekis =len(pav.split())
    zodziu_kiekiai_kauno.append(kiekis)
    
vidutinis_zodziu_kiekis_kauno = sum(zodziu_kiekiai_kauno) / len(zodziu_kiekiai_kauno)
print(f'vidutinis zodziu kiekis "Kauno diena" pavadinimuose: {vidutinis_zodziu_kiekis_kauno}')
print(zodziu_kiekiai_kauno)


vidutinis zodziu kiekis "Kauno diena" pavadinimuose: 8.551724137931034
[12, 8, 7, 10, 10, 6, 11, 9, 10, 8, 8, 11, 7, 9, 6, 12, 13, 11, 6, 7, 7, 7, 7, 5, 9, 14, 3, 8, 7]


In [12]:
SDB = sqlite3.connect('Testas.db')  # jei neegzistuoja db , bus sukurta nauja db
Cs = SDB.cursor()

#  if not exists - tikrina ar jau sukurta DB
sql = '''create table if not exists KaunoDiena
(
pavadinimas text not null,
nuoroda text not null,
zodziu_kiekis integer not null
)
'''
Cs.execute(sql)

# sql_template = '''insert into T1 values (?,?,?)'''
# for a, b, c in zip(pavadinimai, nuorodos, zodziukiekiai):
#     Cs.execute(sql_template, (a, b, c))

sql_template = '''insert into KaunoDiena values (?,?,?)'''
data = list(zip(pavadinimai_kauno, nuorodos_kauno, zodziu_kiekiai_kauno))
Cs.executemany(sql_template, data)


SDB.commit() # commit() butinas jei norim irasyti duomenis i DB

SDB.close()

In [19]:
SDB = sqlite3.connect('Testas.db')  # jei neegzistuoja db , bus sukurta nauja db
Cs = SDB.cursor()
sql = '''
select * from KaunoDiena;
'''
Cs.execute(sql)
SDB.commit() # commit() butinas jei norim irasyti duomenis i DB

df = pd.read_sql_query(sql, con=SDB)
df.head()
# SDB.close()


,pavadinimas,nuoroda,zodziu_kiekis
0,"2024-ųjų Seimo rinkimai: laimėjo socdemai, ant...",naujienos/lietuva/politika/lietuva-renka-nauja...,12
1,Į antrąjį Seimo rinkimų turą patekę kandidatai...,naujienos/lietuva/politika/laukia-kova-vienman...,8
2,Judrioje Aleksoto gatvėje – laikini eismo ribo...,naujienos/kaunas/miesto-pulsas/judrioje-alekso...,7
3,Septyniose apygardose bus perskaičiuoti balsai...,naujienos/lietuva/politika/sesiose-apygardose-...,10
4,Kauniečiai noriai balsavo Seimo rinkimuose: ka...,naujienos/kaunas/miesto-pulsas/kaunieciai-nori...,10


In [20]:
# autoplius

opcijos = Options()
opcijos.add_argument('--incognito')
driver = webdriver.Chrome(options=opcijos)
url = 'https://autoplius.lt/skelbimai/naudoti-automobiliai'
driver.get(url)
time.sleep(5)
source = driver.page_source
driver.close()

In [34]:
bs = BeautifulSoup(source, 'html.parser')
# print(bs)
auto_nuorodos = []


auto_links = bs.find('div', {'class':'auto-lists lt'}).find_all('a')
for link in auto_links:
    # print(link['href'])
    auto_nuorodos.append(link['href'])

print(auto_nuorodos)
print(len(auto_nuorodos))

kitas = bs.find('div', {'class':'page-navigation-container'}).find('a', {'class':'next'})
print(kitas['href'])

next_nuoroda = f'https://autoplius.lt{kitas['href']}'
print(next_nuoroda)

['https://autoplius.lt/skelbimai/bmw-428-gran-coupe-2-0-l-kupe-coupe-2015-benzinas-26837939.html', 'https://autoplius.lt/skelbimai/bmw-530-2-0-l-sedanas-2019-benzinas-26808537.html', 'https://autoplius.lt/skelbimai/bmw-530-2-0-l-sedanas-2021-benzinas-26749453.html', 'https://autoplius.lt/skelbimai/bmw-530-3-0-l-sedanas-2004-benzinas-26675965.html', 'https://autoplius.lt/skelbimai/fiat-500x-2-4-l-visureigis-krosoveris-2016-benzinas-dujos-26862175.html', 'https://autoplius.lt/skelbimai/bmw-320-gran-turismo-2-0-l-hecbekas-2016-dyzelinas-26830873.html', 'https://autoplius.lt/skelbimai/mazda-cx-5-2-0-l-visureigis-krosoveris-2017-benzinas-26740367.html', 'https://autoplius.lt/skelbimai/mercedes-benz-clk320-3-2-l-kupe-coupe-2002-benzinas-dujos-26445796.html', 'https://autoplius.lt/skelbimai/bmw-520-2-0-l-universalas-2006-dyzelinas-26806059.html', 'https://autoplius.lt/skelbimai/volkswagen-amarok-2-0-l-pikapas-2014-dyzelinas-26615729.html', 'https://autoplius.lt/skelbimai/land-rover-discovery-

In [36]:
for nuoroda in auto_nuorodos[0:1]:
    print(nuoroda)
    driver = webdriver.Chrome(options=opcijos)
    url = nuoroda
    driver.get(url)
    time.sleep(5)
    source = driver.page_source
    driver.close()
    
    

https://autoplius.lt/skelbimai/bmw-428-gran-coupe-2-0-l-kupe-coupe-2015-benzinas-26837939.html


In [40]:
bs_auto = BeautifulSoup(source, 'html.parser')

car = bs_auto.find('ol', {'class':'breadcrumbs'}).find_all('li', {'class':'crumb'})
marke = car[2].text.strip()
modelis = car[3].text.strip()
print(marke, modelis)

BMW 428 Gran Coupe
